In [41]:
from joblib import Parallel, delayed
from tqdm import tqdm

import numpy as np
import polars as pl
from scipy.linalg import sqrtm
from scipy.stats import chi2
from typing import Optional
from lrv_test import LRV


import pickle
from pathlib import Path
from typing import Callable

from gpy_test import GPY

from data_generation.arma import (
    generate_arma_sample,
    generate_arma_covariance_matrix,
)

from data_generation.other import (
    factor_model_sample,
    common_random_dependence_sample,
)

from joblib import Parallel, delayed
from tqdm import tqdm

import numpy as np
import polars as pl
from lrv_test import LRV
from scipy.linalg import sqrtm

import pickle
from pathlib import Path
from typing import Callable

from typing import Optional
from data_generation.arma import generate_arma_sample, generate_ar_covariance_matrix
from data_generation.spectral_density import _arma_spectral_density

from data_generation.other import factor_model_sample, common_random_dependence_sample

# Default values

In [42]:
# Test function used for both LRV and GPY test
f = lambda x: (x - 1) ** 2

# Regime
N_range = np.linspace(1000, 8000, 8)
alpha = 0.66
c = 1 / 2

# Used for the LRV test in the lag window estimator
gamma = 1 / 4  # L = int(N**gamma)


def get_freqs(N: int, B: int) -> np.ndarray:
    # frequencies to evaluate, exclude -0.5 and 0 as they
    # have not well behaved: big estimation error at these point
    # for spectral density
    return np.arange(-0.4, 0.6, (B / N) * 1.1)


# default values for the iid time series. We choose a small AR as that
# creates a very spiky spectral density if too large
time_ar, time_ma = 0.1, 0.5

# always repeat an experiment K times
n_repeats = 10**4

# threshold to detect. Use a double sided alternative. That helps
# a bit GPY test. df=1 as there is only one function
level = 0.1
chi2_values = chi2.ppf(np.array([level / 2, 1 - level / 2]), df=1)

# generate real or complex gaussian data
is_complex_gaussian = True

# various computation (complex contour integral) should result in a
# real value. We check that the imaginary part is smaller than this
tolerance = 1e-5

# path to the stored experiment results for easy reloading
data_storage_path = "./data/{experiment_type}/{experiment_id}.pickle"

# Batch tools

In [43]:
# Define the functions to store and load experiment results
def run_batch(
    run: Callable, n_repeats: int, experiment_type: str, run_kwargs: dict
) -> None:
    c = run_kwargs["M"] / run_kwargs["B"]

    from lrv_test.contour import Contour
    from lrv_test.functions import support_MP, t
    from lrv_test.sigma import compute_sigma
    from lrv_test.utils import action_D_on_f, contour_integral, psi

    f_against_mp = action_D_on_f(f, lambda z: t(z, c), support_MP(c), tolerance)
    support = (-np.sqrt(c), np.sqrt(c))
    radius = (support[1] - support[0]) / 2
    center = (support[0] + support[1]) / 2
    contour = Contour.from_circle_parameters(center, radius)
    f_against_D = contour_integral(
        lambda w: -c / (2 * np.pi * 1j) * f(psi(w, c)) / (w**3), contour
    )
    f_against_D = np.real(f_against_D)
    sigma = compute_sigma(f, c, tolerance)
    precomputed = (f_against_mp, f_against_D, sigma)

    experiment_id = "_".join(
        f"{key}={value}"
        for key, value in run_kwargs.items()
        if isinstance(value, int) or isinstance(value, float) or isinstance(value, str)
    )
    resolved_data_path = Path(
        data_storage_path.format(
            experiment_type=experiment_type, experiment_id=experiment_id
        )
    )
    if resolved_data_path.exists():
        return
        # pass
    print(f"Running {experiment_id}")

    resolved_data_path.parent.mkdir(parents=True, exist_ok=True)

    results = [run(precomputed, **run_kwargs) for _ in range(n_repeats)]
    with open(resolved_data_path, "wb") as handle:
        pickle.dump(results, handle)


def load_batch(experiment_type: str) -> pl.DataFrame:
    # load the samples
    all_results = []

    resolved_data_path = Path(
        data_storage_path.format(experiment_type=experiment_type, experiment_id="none")
    ).parent
    for file in resolved_data_path.iterdir():
        if file.stem == ".DS_Store":
            continue

        with open(file, "rb") as handle:
            results = pickle.load(handle)
        all_results += results
        
    df = pl.DataFrame(all_results)
    return df

# Level check

In [44]:
def run(
    precomputed,
    N,
    M,
    B,
    L,
    c,
    time_ar,
    time_ma,
    is_complex_gaussian,
    coef_type: Optional[str] = None,
    oracle_sd: Optional[Callable] = None,
):
    if coef_type == 'constant':
        time_ar, time_ma = time_ar, time_ma 
    else: 
        time_ar, time_ma = np.random.uniform(-0.5, 0.5, M), np.random.uniform(-0.5, 0.5, M)
        
    # f_against_mp, f_against_D, sigma = precomputed
    y = generate_arma_sample(
        N,
        M,
        time_ar,
        time_ma,
        is_complex_gaussian,
    )

    freqs = get_freqs(N, B)
    lrv_result = LRV(
        y,
        B,
        f,
        freqs,
        L,
        sd=oracle_sd,
        # f_against_mp=f_against_mp,
        # f_against_D=f_against_D,
        # sigma=sigma,
    )
    
    gpy_result = GPY(y, [f], is_complex_gaussian=is_complex_gaussian)

    d = {
        "N": N,
        "M": M,
        "B": B,
        "L": L,
        "c": c,
        # "time_ar": time_ar,
        # "time_ma": time_ma,
        "is_complex_gaussian": is_complex_gaussian,
        "t_stat_1": lrv_result.t_stat_1,
        "t_stat_2": lrv_result.t_stat_2,
        "t_stat_3": lrv_result.t_stat_3,
        "t_stat_4": lrv_result.t_stat_4,
        "is_positive_1": lrv_result.is_positive_1(level),
        "is_positive_2": lrv_result.is_positive_2(level),
        "is_positive_3": lrv_result.is_positive_3(level),
        "is_positive_4": lrv_result.is_positive_4(level),
        "gpy_t_stat": gpy_result.test_statistic,
        "is_positive_5": gpy_result.is_positive(level),
        "coef_type": coef_type,
    }
    return d

### Constant vs Random ARMA parameter

In [45]:
tasks = []
for coef_type in ['constant', 'random']:
    for N in N_range:
        L = int(N**gamma)
        B = int(N**alpha)
        if B % 2 == 0:
            B -= 1
        M = int(B * c)

        if coef_type == 'constant':
            ar, ma = time_ar, time_ma 
        else: 
            ar, ma = np.random.uniform(-0.5, 0.5, M), np.random.uniform(-0.5, 0.5, M)

        task = (
            run,
            n_repeats,
            f"arma_no_dependence",
            {
                "N": int(N),
                "M": int(M),
                "B": int(B),
                "L": int(L),
                "c": c,
                "time_ar": ar,
                "time_ma": ma,
                "is_complex_gaussian": is_complex_gaussian,
                'coef_type': coef_type,
            },
        )

        tasks.append(task)

# shuffle the tasks to distribute the load over time
np.random.shuffle(tasks)
results = Parallel(n_jobs=8)(delayed(run_batch)(*task) for task in tqdm(tasks))

df = load_batch(f"arma_no_dependence")
df = df.unpivot(
    index=["N", "M", "B", "L", "c", "is_complex_gaussian", "coef_type"],
    on=[f"is_positive_{i}" for i in range(1, 6)],
    value_name="is_positive",
    variable_name="test",
)
pivot = (
    df
    .group_by(["N", "test", "coef_type"])
    .mean()
    .sort(by=["N", "coef_type", "test"])
    .to_pandas()
    .pivot(index="N", columns=['coef_type', "test"], values="is_positive")
)
print(pivot.to_latex(float_format="%.3f"))

Running N=2000_M=74_B=149_L=6_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
Running N=4000_M=118_B=237_L=7_c=0.5_is_complex_gaussian=True_coef_type=random
Running N=6000_M=155_B=311_L=8_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
Running N=2000_M=74_B=149_L=6_c=0.5_is_complex_gaussian=True_coef_type=random
Running N=3000_M=98_B=197_L=7_c=0.5_is_complex_gaussian=True_coef_type=random
Running N=8000_M=187_B=375_L=9_c=0.5_is_complex_gaussian=True_coef_type=random
Running N=8000_M=187_B=375_L=9_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
Running N=1000_M=47_B=95_L=5_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant


100%|██████████| 16/16 [43:08<00:00, 161.80s/it]


Running N=1000_M=47_B=95_L=5_c=0.5_is_complex_gaussian=True_coef_type=random
Running N=4000_M=118_B=237_L=7_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
Running N=7000_M=171_B=343_L=9_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
Running N=3000_M=98_B=197_L=7_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
Running N=6000_M=155_B=311_L=8_c=0.5_is_complex_gaussian=True_coef_type=random
Running N=5000_M=137_B=275_L=8_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
Running N=5000_M=137_B=275_L=8_c=0.5_is_complex_gaussian=True_coef_type=random
Running N=7000_M=171_B=343_L=9_c=0.5_is_complex_gaussian=True_coef_type=random
\begin{tabular}{lrrrrrrrrrr}
\toprule
coef_type & \multicolumn{5}{r}{constant} & \multicolumn{5}{r}{random} \\
test & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 & is_positive_5 & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 & is_po

In [46]:
pivot

coef_type      constant                                            \
test      is_positive_1 is_positive_2 is_positive_3 is_positive_4   
N                                                                   
1000             0.1374        0.1186        0.1527        0.1092   
2000             0.1313        0.1086        0.1338        0.0897   
3000             0.1062        0.0953        0.1217        0.0921   
4000             0.1164        0.0978        0.1191        0.0907   
5000             0.1118        0.0957        0.1151        0.0842   
6000             0.1106        0.0965        0.1138        0.0830   
7000             0.1121        0.0973        0.1132        0.0846   
8000             0.1110        0.0934        0.1125        0.0828   

coef_type                      random                              \
test      is_positive_5 is_positive_1 is_positive_2 is_positive_3   
N                                                                   
1000             0.1040        0.1128        0.0821        0.0982   
2000             0.0999        0.1140        0.0869        0.0958   
3000             0.0972        0.0950        0.0773        0.0881   
4000             0.1020        0.1051        0.0817        0.0932   
5000             0.1060        0.1024        0.0813        0.0920   
6000             0.1031        0.1014        0.0810        0.0918   
7000             0.0953        0.1074        0.0853        0.0931   
8000             0.0959        0.1009        0.0848        0.0915   

coef_type                              
test      is_positive_4 is_positive_5  
N                                      
1000             0.0783        0.6136  
2000             0.0725        0.6937  
3000             0.0700        0.7339  
4000             0.0787        0.7565  
5000             0.0740        0.7734  
6000             0.0753        0.7798  
7000             0.0693        0.7895  
8000             0.0705        0.8017

# Power check

## AR covariance structure

In [15]:
def run(
    precomputed,
    N,
    M,
    B,
    L,
    c,
    time_ar,
    time_ma,
    space_ar,
    is_complex_gaussian,
):
    f_against_mp, f_against_D, sigma = precomputed
    y = generate_arma_sample(
        N,
        M,
        time_ar,
        time_ma,
        is_complex_gaussian,
    )
    Cov = generate_ar_covariance_matrix(M, space_ar)
    y = y @ sqrtm(Cov)

    freqs = get_freqs(N, B)
    lrv_result = LRV(
        y,
        B,
        f,
        freqs,
        L,
        f_against_mp=f_against_mp,
        f_against_D=f_against_D,
        sigma=sigma,
    )

    gpy_result = GPY(y, [f], is_complex_gaussian=is_complex_gaussian)

    return {
        "N": N,
        "M": M,
        "B": B,
        "L": L,
        "c": c,
        "time_ar": time_ar,
        "time_ma": time_ma,
        "space_ar": space_ar,
        "is_complex_gaussian": is_complex_gaussian,
        "t_stat_1": lrv_result.t_stat_1,
        "t_stat_2": lrv_result.t_stat_2,
        "t_stat_3": lrv_result.t_stat_3,
        "t_stat_4": lrv_result.t_stat_4,
        "is_positive_1": lrv_result.is_positive_1(level),
        "is_positive_2": lrv_result.is_positive_2(level),
        "is_positive_3": lrv_result.is_positive_3(level),
        "is_positive_4": lrv_result.is_positive_4(level),
        "gpy_t_stat": gpy_result.test_statistic,
        "is_positive_5": gpy_result.is_positive(level),
    }

In [16]:
# ARMA covariance structure
space_ar_range = [0.05, 0.5]

tasks = []
for space_ar in space_ar_range: 
    for N in N_range:
        L = int(N**gamma)
        B = int(N**alpha)
        if B % 2 == 0:
            B -= 1
        M = int(B * c)

        task = (
            run,
            n_repeats,
            f"ar_dependence",
            {
                "N": int(N),
                "M": int(M),
                "B": int(B),
                "L": int(L),
                "c": c,
                "time_ar": time_ar,
                "time_ma": time_ma,
                "space_ar": space_ar,
                "is_complex_gaussian": is_complex_gaussian,
            },
        )

        tasks.append(task)

# shuffle the tasks to distribute the load over time
np.random.shuffle(tasks)
results = Parallel(n_jobs=8)(delayed(run_batch)(*task) for task in tqdm(tasks))

df = load_batch(f"ar_dependence").drop(["time_ar", "time_ma"])
df = df.unpivot(
    index=["N", "M", "B", "L", "c", "is_complex_gaussian", 'space_ar'],
    on=[f"is_positive_{i}" for i in range(1, 6)],
    value_name="is_positive",
    variable_name="test",
)

Running N=2000_M=74_B=149_L=6_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.5_is_complex_gaussian=True
Running N=7000_M=171_B=343_L=9_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True
Running N=3000_M=98_B=197_L=7_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True
Running N=6000_M=155_B=311_L=8_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True
Running N=4000_M=118_B=237_L=7_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True
Running N=5000_M=137_B=275_L=8_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.5_is_complex_gaussian=True
Running N=2000_M=74_B=149_L=6_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True
Running N=4000_M=118_B=237_L=7_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.5_is_complex_gaussian=True


100%|██████████| 16/16 [51:07<00:00, 191.72s/it]

Running N=8000_M=187_B=375_L=9_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True


Running N=7000_M=171_B=343_L=9_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.5_is_complex_gaussian=True
Running N=1000_M=47_B=95_L=5_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.5_is_complex_gaussian=True


/Users/alexisrosuel/micromamba/envs/lrv312/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Running N=6000_M=155_B=311_L=8_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.5_is_complex_gaussian=True
Running N=5000_M=137_B=275_L=8_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True
Running N=8000_M=187_B=375_L=9_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.5_is_complex_gaussian=True
Running N=3000_M=98_B=197_L=7_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.5_is_complex_gaussian=True
Running N=1000_M=47_B=95_L=5_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True


In [17]:
pivot = (
    df
    .group_by(["N", 'space_ar', "test"])
    .mean()
    .sort(["N", 'space_ar', "test"])
    .to_pandas()
    .pivot(index="N", columns=['space_ar', "test"], values="is_positive")
)
print(pivot.to_latex(float_format="%.3f"))

\begin{tabular}{lrrrrrrrrrr}
\toprule
space_ar & \multicolumn{5}{r}{0.050000} & \multicolumn{5}{r}{0.500000} \\
test & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 & is_positive_5 & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 & is_positive_5 \\
N &  &  &  &  &  &  &  &  &  &  \\
\midrule
1000 & 0.189 & 0.140 & 0.183 & 0.127 & 0.096 & 1.000 & 1.000 & 1.000 & 1.000 & 0.154 \\
2000 & 0.478 & 0.220 & 0.277 & 0.177 & 0.087 & 1.000 & 1.000 & 1.000 & 1.000 & 0.172 \\
3000 & 0.747 & 0.335 & 0.408 & 0.248 & 0.082 & 1.000 & 1.000 & 1.000 & 1.000 & 0.178 \\
4000 & 0.918 & 0.507 & 0.589 & 0.329 & 0.084 & 1.000 & 1.000 & 1.000 & 1.000 & 0.197 \\
5000 & 0.982 & 0.686 & 0.754 & 0.422 & 0.086 & 1.000 & 1.000 & 1.000 & 1.000 & 0.196 \\
6000 & 0.997 & 0.832 & 0.877 & 0.519 & 0.080 & 1.000 & 1.000 & 1.000 & 1.000 & 0.204 \\
7000 & 1.000 & 0.922 & 0.948 & 0.617 & 0.081 & 1.000 & 1.000 & 1.000 & 1.000 & 0.209 \\
8000 & 1.000 & 0.973 & 0.984 & 0.710 & 0.083 & 1.000 & 1.000

In [18]:
pivot

space_ar          0.05                                            \
test     is_positive_1 is_positive_2 is_positive_3 is_positive_4   
N                                                                  
1000            0.1885        0.1396        0.1827        0.1270   
2000            0.4783        0.2199        0.2774        0.1768   
3000            0.7472        0.3347        0.4079        0.2481   
4000            0.9176        0.5069        0.5894        0.3290   
5000            0.9822        0.6861        0.7538        0.4225   
6000            0.9972        0.8323        0.8772        0.5194   
7000            0.9997        0.9218        0.9483        0.6172   
8000            1.0000        0.9731        0.9842        0.7099   

space_ar                        0.50                              \
test     is_positive_5 is_positive_1 is_positive_2 is_positive_3   
N                                                                  
1000            0.0963           1.0           1.0           1.0   
2000            0.0870           1.0           1.0           1.0   
3000            0.0820           1.0           1.0           1.0   
4000            0.0842           1.0           1.0           1.0   
5000            0.0861           1.0           1.0           1.0   
6000            0.0797           1.0           1.0           1.0   
7000            0.0813           1.0           1.0           1.0   
8000            0.0834           1.0           1.0           1.0   

space_ar                              
test     is_positive_4 is_positive_5  
N                                     
1000               1.0        0.1536  
2000               1.0        0.1721  
3000               1.0        0.1782  
4000               1.0        0.1969  
5000               1.0        0.1957  
6000               1.0        0.2044  
7000               1.0        0.2089  
8000               1.0        0.2172

## Common random dependence


In [19]:
def run(
    precomputed,
    N,
    M,
    B,
    L,
    c,
    time_ar,
    time_ma,
    mixing_scale,
    is_complex_gaussian,
):
    f_against_mp, f_against_D, sigma = precomputed

    y = common_random_dependence_sample(
        N, M, time_ar, time_ma, mixing_scale, is_complex_gaussian
    )

    freqs = get_freqs(N, B)
    lrv_result = LRV(
        y,
        B,
        f,
        freqs,
        L,
        f_against_mp=f_against_mp,
        f_against_D=f_against_D,
        sigma=sigma,
    )
    gpy_result = GPY(y, [f], is_complex_gaussian=is_complex_gaussian)
    
    return {
        "N": N,
        "M": M,
        "B": B,
        "L": L,
        "c": c,
        "time_ar": time_ar,
        "time_ma": time_ma,
        "mixing_scale": mixing_scale,
        "is_complex_gaussian": is_complex_gaussian,
        "t_stat_1": lrv_result.t_stat_1,
        "t_stat_2": lrv_result.t_stat_2,
        "t_stat_3": lrv_result.t_stat_3,
        "t_stat_4": lrv_result.t_stat_4,
        "is_positive_1": lrv_result.is_positive_1(level),
        "is_positive_2": lrv_result.is_positive_2(level),
        "is_positive_3": lrv_result.is_positive_3(level),
        "is_positive_4": lrv_result.is_positive_4(level),
        "gpy_t_stat": gpy_result.test_statistic,
        "is_positive_5": gpy_result.is_positive(level),
    }

In [20]:
mixing_scale_range = [0.05, 0.5]

tasks = []
for mixing_scale in mixing_scale_range:
    for N in N_range:
        L = int(N**gamma)
        B = int(N**alpha)
        if B % 2 == 0:
            B -= 1
        M = int(B * c)

        task = (
            run,
            n_repeats,
            "common_random_dependence",
            {
                "N": int(N),
                "M": int(M),
                "B": int(B),
                "L": int(L),
                "c": c,
                "time_ar": time_ar,
                "time_ma": time_ma,
                "mixing_scale": mixing_scale,
                "is_complex_gaussian": is_complex_gaussian,
            },
        )

        tasks.append(task)

# shuffle the tasks to distribute the load over time
np.random.shuffle(tasks)
results = Parallel(n_jobs=8)(delayed(run_batch)(*task) for task in tqdm(tasks))

df = load_batch(f"common_random_dependence")
df = df.unpivot(
    index=["N", "M", "B", "L", "c", "is_complex_gaussian", "mixing_scale" ],
    on=[f"is_positive_{i}" for i in range(1, 6)],
    value_name="is_positive",
    variable_name="test",
)

Running N=2000_M=74_B=149_L=6_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.5_is_complex_gaussian=True
Running N=3000_M=98_B=197_L=7_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True
Running N=3000_M=98_B=197_L=7_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.5_is_complex_gaussian=True
Running N=7000_M=171_B=343_L=9_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.5_is_complex_gaussian=True
Running N=2000_M=74_B=149_L=6_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=TrueRunning N=6000_M=155_B=311_L=8_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.5_is_complex_gaussian=True

Running N=5000_M=137_B=275_L=8_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.5_is_complex_gaussian=True
Running N=8000_M=187_B=375_L=9_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.5_is_complex_gaussian=True


100%|██████████| 16/16 [49:15<00:00, 184.69s/it]


Running N=7000_M=171_B=343_L=9_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True
Running N=1000_M=47_B=95_L=5_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True
Running N=4000_M=118_B=237_L=7_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True
Running N=4000_M=118_B=237_L=7_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.5_is_complex_gaussian=True
Running N=1000_M=47_B=95_L=5_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.5_is_complex_gaussian=True
Running N=6000_M=155_B=311_L=8_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True
Running N=8000_M=187_B=375_L=9_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True
Running N=5000_M=137_B=275_L=8_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True


In [21]:
pivot = (
    df
    .group_by(["N", "test", 'mixing_scale'])
    .mean()
    .sort(["N", 'mixing_scale', "test"])
    .to_pandas()
    .pivot(index="N", columns=['mixing_scale', "test"], values="is_positive")
)
print(pivot.to_latex(float_format="%.3f"))

\begin{tabular}{lrrrrrrrrrr}
\toprule
mixing_scale & \multicolumn{5}{r}{0.050000} & \multicolumn{5}{r}{0.500000} \\
test & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 & is_positive_5 & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 & is_positive_5 \\
N &  &  &  &  &  &  &  &  &  &  \\
\midrule
1000 & 0.182 & 0.148 & 0.189 & 0.136 & 0.111 & 1.000 & 1.000 & 1.000 & 1.000 & 0.172 \\
2000 & 0.479 & 0.220 & 0.274 & 0.182 & 0.110 & 1.000 & 1.000 & 1.000 & 1.000 & 0.183 \\
3000 & 0.744 & 0.340 & 0.414 & 0.249 & 0.110 & 1.000 & 1.000 & 1.000 & 1.000 & 0.182 \\
4000 & 0.913 & 0.508 & 0.587 & 0.341 & 0.114 & 1.000 & 1.000 & 1.000 & 1.000 & 0.187 \\
5000 & 0.980 & 0.686 & 0.751 & 0.423 & 0.113 & 1.000 & 1.000 & 1.000 & 1.000 & 0.184 \\
6000 & 0.996 & 0.823 & 0.872 & 0.520 & 0.113 & 1.000 & 1.000 & 1.000 & 1.000 & 0.184 \\
7000 & 1.000 & 0.921 & 0.947 & 0.618 & 0.109 & 1.000 & 1.000 & 1.000 & 1.000 & 0.184 \\
8000 & 1.000 & 0.967 & 0.979 & 0.698 & 0.113 & 1.000 & 1

In [32]:
pivot

snr           0.05                                                    0.20  \
test is_positive_1 is_positive_2 is_positive_3 is_positive_4 is_positive_1   
N                                                                            
1000        0.1006        0.0650        0.0789        0.0597         0.999   
2000        0.1162        0.0849        0.0997        0.0701         1.000   
3000        0.1053        0.0825        0.0955        0.0757         1.000   
4000        0.1333        0.0906        0.1060        0.0830         1.000   
5000        0.1554        0.0943        0.1137        0.0890         1.000   
6000        0.1806        0.1092        0.1326        0.0989         1.000   
7000        0.2225        0.1244        0.1564        0.1096         1.000   
8000        0.2840        0.1438        0.1855        0.1300         1.000   

snr                                             
test is_positive_2 is_positive_3 is_positive_4  
N                                               
1000        0.9991        0.9995        0.9977  
2000        1.0000        1.0000        1.0000  
3000        1.0000        1.0000        1.0000  
4000        1.0000        1.0000        1.0000  
5000        1.0000        1.0000        1.0000  
6000        1.0000        1.0000        1.0000  
7000        1.0000        1.0000        1.0000  
8000        1.0000        1.0000        1.0000

## Factor model 

In [23]:
def run(
    precomputed,
    N,
    M,
    B,
    L,
    r,
    snr,
    time_ar,
    time_ma,
    is_complex_gaussian,
):
    f_against_mp, f_against_D, sigma = precomputed

    y = factor_model_sample(N, M, r, time_ar, time_ma, snr, is_complex_gaussian)

    freqs = get_freqs(N, B)
    lrv_result = LRV(
        y,
        B,
        f,
        freqs,
        L,
        f_against_mp=f_against_mp,
        f_against_D=f_against_D,
        sigma=sigma,
    )

    gpy_result = GPY(y, [f], is_complex_gaussian=is_complex_gaussian)
    
    return {
        "N": N,
        "M": M,
        "B": B,
        "L": L,
        "r": r,
        "c": c,
        "snr": snr,
        "time_ar": time_ar,
        "time_ma": time_ma,
        "is_complex_gaussian": is_complex_gaussian,
        "t_stat_1": lrv_result.t_stat_1,
        "t_stat_2": lrv_result.t_stat_2,
        "t_stat_3": lrv_result.t_stat_3,
        "t_stat_4": lrv_result.t_stat_4,
        "is_positive_1": lrv_result.is_positive_1(level),
        "is_positive_2": lrv_result.is_positive_2(level),
        "is_positive_3": lrv_result.is_positive_3(level),
        "is_positive_4": lrv_result.is_positive_4(level),
        "gpy_t_stat": gpy_result.test_statistic,
        "is_positive_5": gpy_result.is_positive(level),
    }

In [34]:
snr_range = [0.05, 0.2]
r = 1

tasks = []
for snr in snr_range:
    for N in N_range:
        L = int(N**gamma)
        B = int(N**alpha)
        if B % 2 == 0:
            B -= 1
        M = int(B * c)

        task = (
            run,
            n_repeats,
            f"factor_model",
            {
                "N": int(N),
                "M": int(M),
                "B": int(B),
                "L": int(L),
                "r": r,
                "snr": snr,
                "time_ar": time_ar,
                "time_ma": time_ma,
                "is_complex_gaussian": is_complex_gaussian,
            },
        )

        tasks.append(task)

# shuffle the tasks to distribute the load over time
np.random.shuffle(tasks)
results = Parallel(n_jobs=8)(delayed(run_batch)(*task) for task in tqdm(tasks))

df = load_batch(f"factor_model")
df = df.unpivot(
    index=["N", "M", "B", "L", "c", "snr", "r", "is_complex_gaussian"],
    on=[f"is_positive_{i}" for i in range(1, 6)],
    value_name="is_positive",
    variable_name="test",
)

100%|██████████| 16/16 [00:03<00:00,  5.24it/s]


In [35]:
pivot = (
    df
    .group_by(["N", "test", "snr"])
    .mean()
    .sort(["N", 'snr', "test"])
    .to_pandas()
    .pivot(index="N", columns=["snr", "test"], values="is_positive")
)
pivot = pivot.sort_index(axis=1, level=[0, 1])
print(pivot.to_latex(float_format="%.3f"))

\begin{tabular}{lrrrrrrrrrr}
\toprule
snr & \multicolumn{5}{r}{0.050000} & \multicolumn{5}{r}{0.200000} \\
test & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 & is_positive_5 & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 & is_positive_5 \\
N &  &  &  &  &  &  &  &  &  &  \\
\midrule
1000 & 0.101 & 0.065 & 0.079 & 0.060 & 0.095 & 0.999 & 0.999 & 1.000 & 0.998 & 0.169 \\
2000 & 0.116 & 0.085 & 0.100 & 0.070 & 0.102 & 1.000 & 1.000 & 1.000 & 1.000 & 0.192 \\
3000 & 0.105 & 0.083 & 0.096 & 0.076 & 0.104 & 1.000 & 1.000 & 1.000 & 1.000 & 0.185 \\
4000 & 0.133 & 0.091 & 0.106 & 0.083 & 0.101 & 1.000 & 1.000 & 1.000 & 1.000 & 0.192 \\
5000 & 0.155 & 0.094 & 0.114 & 0.089 & 0.101 & 1.000 & 1.000 & 1.000 & 1.000 & 0.198 \\
6000 & 0.181 & 0.109 & 0.133 & 0.099 & 0.106 & 1.000 & 1.000 & 1.000 & 1.000 & 0.195 \\
7000 & 0.223 & 0.124 & 0.156 & 0.110 & 0.102 & 1.000 & 1.000 & 1.000 & 1.000 & 0.202 \\
8000 & 0.284 & 0.144 & 0.185 & 0.130 & 0.100 & 1.000 & 1.000 & 1.

In [33]:
pivot

snr           0.05                                                    0.20  \
test is_positive_1 is_positive_2 is_positive_3 is_positive_4 is_positive_1   
N                                                                            
1000        0.1006        0.0650        0.0789        0.0597         0.999   
2000        0.1162        0.0849        0.0997        0.0701         1.000   
3000        0.1053        0.0825        0.0955        0.0757         1.000   
4000        0.1333        0.0906        0.1060        0.0830         1.000   
5000        0.1554        0.0943        0.1137        0.0890         1.000   
6000        0.1806        0.1092        0.1326        0.0989         1.000   
7000        0.2225        0.1244        0.1564        0.1096         1.000   
8000        0.2840        0.1438        0.1855        0.1300         1.000   

snr                                             
test is_positive_2 is_positive_3 is_positive_4  
N                                               
1000        0.9991        0.9995        0.9977  
2000        1.0000        1.0000        1.0000  
3000        1.0000        1.0000        1.0000  
4000        1.0000        1.0000        1.0000  
5000        1.0000        1.0000        1.0000  
6000        1.0000        1.0000        1.0000  
7000        1.0000        1.0000        1.0000  
8000        1.0000        1.0000        1.0000

In [31]:
import numpy as np

10 * np.log10(0.05), 10 * np.log10(0.20)

(np.float64(-13.010299956639813), np.float64(-6.9897000433601875))